In [ ]:
import sys
sys.path.append("../input/tez-lib/")

In [ ]:
import torch

import pandas as pd
import torch.nn as nn

from scipy import stats
from tez import Tez, TezConfig
from tez.callbacks import EarlyStopping
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup

In [ ]:
class args:
    model = "../input/anferico-bert-for-patents/"
    max_len = 32
    accumulation_steps = 1
    batch_size = 64
    epochs = 5
    learning_rate = 2e-5

In [ ]:
class PhraseDataset:
    def __init__(self, anchor, target, context, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]

        encoded_text = self.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]
        token_type_ids = encoded_text["token_type_ids"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }

In [ ]:
class PhraseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name

        config = AutoConfig.from_pretrained(model_name)
        config.update(
            {
                "output_hidden_states": True,
                "add_pooling_layer": True,
                "num_labels": 1,
            }
        )
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.output = nn.Linear(config.hidden_size, 1)

    def forward(self, ids, mask, token_type_ids):
        transformer_out = self.transformer(ids, mask, token_type_ids)
        output = transformer_out.pooler_output
        output = self.dropout(output)
        output = self.output(output)
        return output, 0, {}

In [ ]:
df = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")

context_mapping = {
    "A": "Human Necessities",
    "B": "Operations and Transport",
    "C": "Chemistry and Metallurgy",
    "D": "Textiles",
    "E": "Fixed Constructions",
    "F": "Mechanical Engineering",
    "G": "Physics",
    "H": "Electricity",
    "Y": "Emerging Cross-Sectional Technologies",
}

df.context = df.context.apply(lambda x: context_mapping[x[0]])

tokenizer = AutoTokenizer.from_pretrained(args.model)
test_dataset = PhraseDataset(
    anchor=df.anchor.values,
    target=df.target.values,
    context=df.context.values,
    tokenizer=tokenizer,
    max_len=args.max_len,
)

model = PhraseModel(model_name=args.model)
model = Tez(model)
model_path = "../input/uspppm-tez-models/model_f0.bin"
config = TezConfig(
    test_batch_size=64,
    device="cuda",
)
model.load(model_path, weights_only=True, config=config)

preds_iter = model.predict(test_dataset)
final_preds = []
for preds in preds_iter:
    preds[preds < 0] = 0
    preds[preds > 1] = 1
    final_preds.extend(preds.ravel().tolist())

In [ ]:
sample_submission = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/sample_submission.csv")
sample_submission.score = final_preds
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission.head()